In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import model_selection
import numpy as np
import re
import pickle
from sklearn.ensemble import RandomForestRegressor
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from time import sleep
import schedule
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import model_selection
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import accuracy_score 
import lxml.html
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import schedule



In [ ]:
options = Options() 
options.add_argument('--headless')
options.add_argument('--disable-gpu')             
options.add_argument('--disable-extensions')       
options.add_argument('--proxy-server="direct://"') 
options.add_argument('--proxy-bypass-list=*')      
options.add_argument('--start-maximized')      
driver = Chrome(executable_path="", options=options)

raceDict = {}
timeList = []
url = "https://race.netkeiba.com/top/"

driver.get(url)
sleep(1)

def getTime(date):
    soup = BeautifulSoup(table[0].get_attribute('outerHTML'), 'html.parser')
    sleep(1)
    raceDate = soup.select('.RaceList_Itemtime')
    if not raceDate:
        return []
    return raceDate[0].text.replace('[<span class="RaceList_Itemtime">', '').replace('\n</span>]', '')

# TODO 開催場が3つとは限らないため改善が必要
for j in tqdm(range(1,4)):
    for i in range(1,13):
        '''
        発走時刻の取得
        '''
        table = driver.find_elements(By.XPATH,'//*[@id="RaceTopRace"]/div/dl[' + str(j) +']/dd/ul/li[' + str(i) +']/a[1]/div[2]/div[2]/span[1]' )
        raceDate = getTime(table[0])
        
        # Win5の影響で取得するpathが変わるため
        if not raceDate:
            sleep(1)
            table = driver.find_elements(By.XPATH,'//*[@id="RaceTopRace"]/div/dl[' + str(j) +']/dd/ul/li[' + str(i) +']/a[1]/div[2]/div[2]/span[2]' )
            raceDate = getTime(table[0])
            
        # 末尾の空白削除
        raceDate = raceDate.rstrip()
        # 文字列をdatetimeオブジェクトに変換
        raceDate = datetime.strptime(raceDate, '%H:%M')
        # 20分前の時刻を計算
        raceDate = raceDate - timedelta(minutes=20)
        # 新しい時刻を文字列に変換
        raceDate = raceDate.strftime('%H:%M')
        timeList.append(raceDate)
        

        '''
        レースIDの取得
        '''
        sleep(1)
        table = driver.find_elements(By.XPATH,'//*[@id="RaceTopRace"]/div/dl[' + str(j) +']/dd/ul/li[' + str(i) +']/a[1]' )
        text = re.sub( r'\D+', '', str(table[0].get_attribute('href')))

        #[発走時刻 : レースID]の辞書型生成
        raceDict[raceDate] = text

        # 時刻を発走順にソート
        timeList.sort()
    

In [20]:

schedule.clear()
def task(id):
    print(id)
    print('raceScrape_jockeyID_horseIDを開始しました。')
    # horse_id, jockey_idを取得
    df = raceScrape_jockeyID_horseID(id)
    print('raceScrape_jockeyID_horseIDが終了しました。')

    print('mainScrapeを開始しました。')
    # horse_idをもとに出遅れ回数などを取得
    results = mainScrape(df)
    df = pd.concat([df, results], axis=1)
    print('mainScrapeが終了しました。')

    # ツイート用に元ネタが必要なのでとっておく
    initial_Value = df.copy()

    print('moldingの取得を開始しました。')
    # エンジニアリングする
    newDf = molding(df)
    print('moldingが終了しました。')

    print('createModelを開始しました。')
    # モデルの作成
    # TODO もともと作成したモデルを使用してもよいのか？
    model = createModel()
    print('createModelが終了しました。')
    
    print('predictを開始しました。')
    # 予測
    proba = predict(model, initial_Value, newDf)
    print('predictが終了しました。')

    print('create_netKeiba_comentsを開始しました。')
    # ネット競馬投稿用文字列生成
    netKeibaText = create_netKeiba_coments(proba)
    print('create_netKeiba_comentsが終了しました。')

    print('postCommentを開始しました。')
    # ネット競馬にコメント投稿
    postComment(netKeibaText, id)
    print('postCommentが終了しました。')

    print('createTweetを開始しました。')
    # ツイッター投稿用文字列生成
    tweetText = createTweet(id, proba)
    print('createTweetが終了しました。')

    print('tweetを開始しました。')
    # ツイッター投稿
    tweet(tweetText)
    print('tweetが終了しました。')
    
    


for t in timeList:
    schedule.every().day.at(t).do(task, raceDict[t])

# 指定された時刻まで待機する
while True:
    schedule.run_pending()
    time.sleep(1)
    now = time.strftime('%H:%M')
    if now > timeList[-1]:
        break

In [7]:
def raceScrape_jockeyID_horseID(id):
    options = Options() 
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')             
    options.add_argument('--disable-extensions')       
    options.add_argument('--proxy-server="direct://"') 
    options.add_argument('--proxy-bypass-list=*')      
    options.add_argument('--start-maximized')      
    driver = Chrome(executable_path="", options=options)
    url = 'https://race.netkeiba.com/race/shutuba.html?race_id=' + id
    df = pd.read_html(url)[0]

    # カラムが二重なので取り除く
    newCol = []
    for i in df.columns:
        newCol.append(i[0])
    df.columns = newCol

    
    target_url = url
    driver.get(target_url)
    sleep(1)

    table = driver.find_elements(By.XPATH,'//*[@id="page"]/div[3]/div[2]/table/tbody/tr')
    horse_jockey_id = []
    for i in range(1,len(table)+1):
        result_id = []

        # horse_id
        sleep(1)
        horse_id = driver.find_elements(By.XPATH,'//*[@id="page"]/div[3]/div[2]/table/tbody/tr[' + str(i) +']/td[4]')
        horse_id = re.sub( r'\D+', '', horse_id[0].get_attribute('outerHTML').split('/horse')[1].split('title')[0])
        result_id.append(horse_id)

        # jockey_id
        sleep(1)
        jockey_id = driver.find_elements(By.XPATH,'//*[@id="page"]/div[3]/div[2]/table/tbody/tr[' + str(i) +']/td[7]')
        jockey_id = re.sub( r'\D+', '', jockey_id[0].get_attribute('outerHTML').split('/recent')[1])
        result_id.append(jockey_id)

        horse_jockey_id.append(result_id)  

    df = pd.concat([df, pd.DataFrame(horse_jockey_id,columns=['horse_id', 'jockey_id'])], axis=1)
    driver.quit()
    return df

In [8]:
def mainScrape(df):
    options = Options() 
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')             
    options.add_argument('--disable-extensions')       
    options.add_argument('--proxy-server="direct://"') 
    options.add_argument('--proxy-bypass-list=*')      
    options.add_argument('--start-maximized')      
    driver = Chrome(executable_path="", options=options)
    horse_id_list = df['horse_id']
    target_url = "https://regist.netkeiba.com/account/?pid=login"
    driver.get(target_url)
    sleep(1)

    USERNAME = ""
    PASSWORD = ""

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[1]/input')
    username_input[0].send_keys(USERNAME)
    sleep(1)

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[2]/input')
    username_input[0].send_keys(PASSWORD)
    sleep(1)

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/div[1]/input')
    username_input[0].click()
    sleep(1)

    deokure_count_detail = {}
    for horse_id in tqdm(horse_id_list):
        try:
            sleep(1)
            rece = driver.get('https://db.netkeiba.com/horse/' + horse_id)
            sleep(1)
            table = driver.find_elements(By.XPATH,'//*[@id="contents"]/div[4]/div/table')
            html = table[0].get_attribute('outerHTML')

            deokureCount = 0
            for i in range(len(pd.read_html(html)[0])):
                if '出遅れ' == pd.read_html(html)[0].loc[[i]]['備考'][i]:
                    deokureCount = deokureCount + 1

            horse_detail_list = []
            horse_detail = []
            horse_detail.append(deokureCount)
            horse_detail.append(len(pd.read_html(html)[0]))
            horse_detail_list.append(horse_detail)
            deokure_count_detail[horse_id] = pd.DataFrame(horse_detail_list,columns=['deokureNum','raceNum'])
        except Exception:
            continue

    for key in deokure_count_detail:
        deokure_count_detail[key].index = [key] * len(deokure_count_detail[key])
    results = pd.concat([deokure_count_detail[key] for key in deokure_count_detail], sort=False)
    results = results.reset_index()
    driver.quit()
    return results
    

In [9]:
def molding(df):
    df['性別'] = df['性齢'].str[0]
    df['年齢'] = df['性齢'].str[1:3]
    df['年齢'] = df['年齢'].astype(int)
    df.drop('性齢', axis=1,inplace=True)
    result = []
    for i  in df['馬体重(増減)'].str[4:].str.replace(')','', regex=True).str[0]:
        if i=='+':
            result.append(1)
        elif i=='-':
            result.append(2)
        else:
            result.append(0)

    df['加減'] = result
    df['体重増減'] = df['馬体重(増減)'].str[4:].str.replace(')','', regex=True).str[1:]
    df.loc[(df.体重増減 == ''), '体重増減'] = '0'
    df.loc[(df.体重増減 == '計不'), '体重増減'] = '0'
    df['体重増減'] = df['体重増減'].astype(int)
    df['馬体重'] = df['馬体重(増減)'].str[0:3]
    # df.loc[(df.馬体重 !='計不'), ['馬体重']].astype(float).mean().round().astype(int)
    df.loc[(df.馬体重=='計不'), '馬体重'] = '471'
    df.loc[(df.馬体重=='--'), '馬体重'] = '471'
    df['馬体重'] = df['馬体重'].astype(int)

    df.drop(['印', '馬名','枠','人気', '騎手', 'お気に入り馬', 'Unnamed: 9_level_0', '厩舎', '馬体重(増減)','index'], axis=1, inplace=True)

    df['horse_id'] = df['horse_id'].astype(int)
    df['jockey_id'] = df['jockey_id'].astype(int)

    dummy = df[['性別']]
    dummy = pd.get_dummies(dummy)
    newDf = pd.concat([df, dummy], axis=1)
    newDf.drop(['性別'],axis=1,inplace=True)

    newDf = newDf[['jockey_id', 'deokureNum', 'raceNum', '馬体重', '体重増減', '馬番', '斤量', '加減']]

    return newDf




In [10]:
def createModel():

    df1 = pd.read_pickle('')
    df2 = pd.read_pickle('')
    df3 = pd.read_pickle('')
    df4 = pd.read_pickle('')
    df = pd.concat([df1,df2,df3],axis=1)
    df = df.merge(df4, on='horse_id')

    df.drop(['タイム', '着差', '着順', '馬名', 'ﾀｲﾑ指数', '通過', '上り', '調教ﾀｲﾑ', '厩舎ｺﾒﾝﾄ', '賞金(万円)','調教師','枠番','人気','馬主', '騎手', '単勝'], axis=1, inplace=True)

    df = df.drop_duplicates(subset=df.columns)

    result = []
    for i in  df['time']:
        try:
            appendResult = i.split(':')[0]
            appendResult = re.sub( r'\D+', '', appendResult)
            if(appendResult == ''):
                result.append(0)
                continue
            result.append(appendResult)
        except IndexError:
            result.append(0)
            continue
            

    df['timeHour'] = result
    df.drop('time', axis=1, inplace=True)

    result = []
    for i in df['situation']:
        if(i == "良"):
            result.append(0)
        elif(i=="稍重"):
            result.append(1)
        elif(i=="重"):
            result.append(2)
        else:
            result.append(3)
    df['situation'] = result

    result = []
    for i in df['whether']:
        if(i == "晴"):
            result.append(0)
        elif(i=="曇"):
            result.append(1)
        elif(i=="雨"):
            result.append(2)
        else:
            result.append(3)
    df['whether'] = result

    result = []
    for i in df['distance']:
        result.append(re.sub( r'\D+', '', i))
    df['distance'] = result
    df['distance'] = df['distance'].astype(int)

    df.loc[(df.rightORleft=='芝'), 'rightORleft'] = np.nan

    result = []
    for i in df['track']:
        if(i == "ダ"):
            result.append(0)
        elif(i=="芝"):
            result.append(1)
        else:
            result.append(2)
    df['track'] = result

    df['性別'] = df['性齢'].str[0]
    df['年齢'] = df['性齢'].str[1:3]
    df['年齢'] = df['年齢'].astype(int)
    df.drop('性齢', axis=1,inplace=True)

    result = []
    for i  in df['馬体重'].str[4:].str.replace(')','', regex=True).str[0]:
        if i=='+':
            result.append(1)
        elif i=='-':
            result.append(2)
        else:
            result.append(0)

    df['加減'] = result
    df['体重増減'] = df['馬体重'].str[4:].str.replace(')','', regex=True).str[1:]
    df.loc[(df.体重増減 == ''), '体重増減'] = '0'
    df['体重増減'] = df['体重増減'].astype(int)
    df['馬体重'] = df['馬体重'].str[0:3]
    # df.loc[(df.馬体重 !='計不'), ['馬体重']].astype(float).mean().round().astype(int)
    df.loc[(df.馬体重=='計不'), '馬体重'] = '471'
    df['馬体重'] = df['馬体重'].astype(int)


    result = []

    for i in df['備考']:
        if i=='出遅れ':
            result.append(1)
        else:
            result.append(0)

    df['出遅れ'] = result
    df.drop('備考', axis=1, inplace=True)

    df['timeHour'] = df['timeHour'].astype(int)
    dummy = df[['timeHour','性別','rightORleft']]
    dummy = pd.get_dummies(dummy)
    dummy = pd.concat([dummy, df[['斤量','馬体重', '馬番','馬体重','体重増減','加減','年齢']]], axis=1)

    known_tansyo = dummy[dummy.timeHour != 0].values  
    unknown_tansyo = dummy[dummy.timeHour == 0].values


    X = known_tansyo[:, 1:]  
    y = known_tansyo[:, 0]

    rfr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
    rfr.fit(X, y)

    predictedTansyo = rfr.predict(unknown_tansyo[:, 1::])
    df.loc[(df.timeHour == 0), 'timeHour'] = predictedTansyo.round().astype(int)

    df['rightORleft'].fillna("右", inplace=True)
    df['horse_id'] = df['horse_id'].astype(int)
    df['jockey_id'] = df['jockey_id'].astype(int)
    df['deokureNum'] = df['deokureNum'].astype(int)
    df['raceNum'] = df['raceNum'].astype(int)

    dummy = df[['rightORleft','性別']]
    dummy = pd.get_dummies(dummy)
    oldDf = pd.concat([df, dummy], axis=1)
    oldDf.drop(['rightORleft','性別'],axis=1,inplace=True)

    oldDf = oldDf[['jockey_id', 'deokureNum', 'raceNum', '馬体重', '体重増減', '馬番', '斤量', '加減','出遅れ']]
    X = oldDf.drop('出遅れ',axis=1)
    y = oldDf['出遅れ']
    params =  {'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alpha': 0, 'num_leaves': 127, 'n_estimators': 1000, 'min_child_samples': 30, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 0.9}

    # LightGBMのモデルを定義する
    lgbm_clf = lgb.LGBMClassifier(**params)


    lgbm_clf.fit(X, y)

    return lgbm_clf

In [11]:
def predict(lgbm_clf, initial_Value, newDf):
    y_pred = lgbm_clf.predict_proba(newDf)
    result = pd.DataFrame(y_pred, columns=['出遅れない', '出遅れる'])
    result = pd.merge(result, initial_Value[['馬番', '馬名']],left_index=True, right_index=True).sort_values(by="出遅れる", ascending=False)
    return result

In [12]:
def create_netKeiba_coments(result):
    saveStr = 'すみません、失礼します\n'
    saveStr = saveStr + '出遅れ予測AIランキングを発表します！\n\n'

    count = 0
    for i in result.itertuples():
        count = count + 1
        if 0.5 < i.出遅れる:
            saveStr = saveStr  + str(count) + '位 : '+str(i.馬番) + ' ' +i.馬名 + ' 【危険】' + '\n'
        else:
            saveStr = saveStr  + str(count) + '位 : '+str(i.馬番) + ' ' +i.馬名 + '\n'
        if(count == 3): break

    saveStr = saveStr + '\n皆さんの競馬予想のお役に立てれば幸いです！\n'
    saveStr = saveStr + '※危険の文字があったら気を付けてください\n'
    saveStr = saveStr + '出遅れ予測AI - ラテイ - '

    return saveStr

In [13]:
def postComment(saveStr,id):
    options = Options() 
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')             
    options.add_argument('--disable-extensions')       
    options.add_argument('--proxy-server="direct://"') 
    options.add_argument('--proxy-bypass-list=*')      
    options.add_argument('--start-maximized')      
    driver = Chrome(executable_path="", options=options)

    target_url = "https://regist.netkeiba.com/account/?pid=login"
    driver.get(target_url)
    sleep(1)

    USERNAME = ""
    PASSWORD = ""

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[1]/input')
    username_input[0].send_keys(USERNAME)
    sleep(1)

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/ul/li[2]/input')
    username_input[0].send_keys(PASSWORD)
    sleep(1)

    username_input = driver.find_elements(By.XPATH,'//*[@id="contents"]/div/form/div/div[1]/input')
    username_input[0].click()
    sleep(1)

    url = 'https://race.netkeiba.com/race/bbs.html?race_id=' + id
    driver.get(url)

    sleep(1)
    username_input = driver.find_elements(By.XPATH,'//*[@id="display_comment"]')
    sleep(1)
    username_input[0].click()

    sleep(1)
    username_input = driver.find_elements(By.XPATH,'//*[@id="comment"]')
    sleep(1)
    username_input[0].click()

    sleep(1)
    username_input = driver.find_elements(By.XPATH,'//*[@id="race_top"]/div[5]/div/div[2]/div[2]/button')
    try:
        sleep(1)
        username_input[0].click()
    except Exception:
        print('きた')

    sleep(1)
    username_input = driver.find_elements(By.XPATH,'//*[@id="comment"]')
    sleep(1)
    username_input[0].click()
    sleep(1)
    username_input[0].send_keys(saveStr)

    sleep(1)
    username_input = driver.find_elements(By.XPATH,'//*[@id="Comment_Form"]/div[3]/a')
    sleep(1)
    username_input[0].click()
    driver.quit()
    

In [19]:
def createTweet(id, result):
    tweet = ''
    racePlace = str(id)[4:6]
    raceNumber = str(id)[10:12]
    if '0' == raceNumber[0]:
        raceNumber = raceNumber[1]

    racePlaceName = ''
    if(racePlace == '05'):
        tweet = '東京' + ' ' + raceNumber + 'R' + ' 出遅れ予測ランキング！\n\r\n\r'
        racePlaceName = '東京'
    elif(racePlace == '06'):
        ptweet = '中山' + ' ' + raceNumber + 'R' + ' 出遅れ予測ランキング！\n\r\n\r'
        racePlaceName = '中山'
    elif(racePlace == '09'):
        tweet = '阪神' + ' ' + raceNumber + 'R' + ' 出遅れ予測ランキング！\n\r\n\r'
        racePlaceName = '阪神'
    elif(racePlace == '10'):
        tweet = '小倉' + ' ' + raceNumber + 'R' + ' 出遅れ予測ランキング！\n\r\n\r'
        racePlaceName = '小倉'
    count = 0
    for i in result.itertuples():
        if 0.5 < i.出遅れる:
            count = count + 1
            parsent = round(i.出遅れる, 2)
            parsent = int(parsent*100)
            tweet = tweet + str(count) + '位 : '+str(i.馬番) + '番 ' +i.馬名 + ' 【出遅れ率 : ' + str(parsent) + '%】\n\r'
            if(count == 3): break

    if tweet.find('出遅れ率') == -1:
        tweet = tweet + "このレースでは出遅れる馬はいないと予測されました。\n\r"

    tweet = tweet + '\n\r#' + racePlaceName + raceNumber + 'R' + '  #競馬 #出遅れ予測 '

    return tweet


In [18]:
test = '01'
if '0' == test[0]:
    print(test)

01


In [15]:
def tweet(tweet):
    options = Options() 
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')             
    options.add_argument('--disable-extensions')       
    options.add_argument('--proxy-server="direct://"') 
    options.add_argument('--proxy-bypass-list=*')      
    options.add_argument('--start-maximized')      
    driver = Chrome(executable_path="", options=options)
    
    target_url = "https://twitter.com/i/flow/login"
    driver.get(target_url)
    sleep(1)

    userName = driver.find_elements(By.XPATH,'//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[5]/label')
    sleep(2)
    userName[0].send_keys("@keiba_deokureAI")


    clickBtn = driver.find_elements(By.XPATH,'//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]/div/span/span')

    sleep(3)
    clickBtn[0].click()

    sleep(1)
    password = driver.find_elements(By.XPATH, '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label')
    sleep(1)
    password[0].send_keys("")



    sleep(1)
    clickBtn = driver.find_elements(By.XPATH, '//*[@id="layers"]/div/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div')
    sleep(1)
    clickBtn[0].click()

    try:
        sleep(3)
        advertisement = driver.find_elements(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div[1]/div/div/div/div[1]/div/div')
        sleep(1)
        advertisement[0].click()
    except Exception:
        print('広告表示なし')

    sleep(2)
    text = driver.find_elements(By.XPATH, 
        '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div[2]/div[1]/div/div/div/div[2]/div[1]/div/div/div/div/div/div/div/div/div/div/label/div[1]/div')
    sleep(1)
    text[0].click()

    sleep(2)
    text = driver.find_elements(By.XPATH, 
        '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[3]/div/div[2]/div[1]/div/div/div/div[2]/div[1]/div/div/div/div/div/div[2]/div/div/div/div/label/div[1]/div/div/div/div/div/div[2]/div')
    sleep(2)

    # 出力したい文字列を埋め込む
    text[0].send_keys(tweet)

    sleep(2)
    text = driver.find_elements(By.XPATH,'//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[3]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div/div[2]/div[3]/div')
    sleep(2)
    text[0].click()

    driver.quit()